# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import requests
from io import BytesIO, StringIO
import contextily as ctx
from IPython.display import HTML, display, IFrame
import osmnx as ox
import osmium
from osm_runner import Runner
import regex as re
import unicodedata


# Data

In [4]:
# get green areas
green_areas = pd.read_csv('https://raw.githubusercontent.com/SowmyaMadala/WDL-Stage4/main/Data/Attracting%20Population/WDL_Green_Areas_Dataset_V1.txt', 
                          sep ="|", 
                          encoding ="latin-1").rename(columns = {'Number of Museums in influence area' : 'no_of_museums',
                                                                 'Number of parking lots in influence area' : 'no_of_parking_lots',
                                                                'Number of tourist attractions in influence area': 'no_of_tourist_attractions', 
                                                                'Green Zones Daily Average Demand':'gz_daily_avg_demand',
                                                                'Daily average of people in influence area': 'daily_avg_people',
                                                                'Number of buildings in influence area' : 'no_of_buildings',
                                                                'Number of families in influence area': 'no_of_families',
                                                                'Number of people resident in influence area' : 'no_of_people_resident'
                                                                })

green_areas.shape, green_areas.dtypes

((128, 14),
 Green_Zone_name               object
 no_of_museums                  int64
 no_of_parking_lots             int64
 no_of_tourist_attractions      int64
 gz_daily_avg_demand          float64
 Resident                       int64
 daily_avg_people             float64
 no_of_buildings                int64
 no_of_families                 int64
 no_of_people_resident          int64
 perc_senior_pop              float64
 perc_pop_until_19_yo         float64
 POINT_X                      float64
 POINT_Y                      float64
 dtype: object)

In [5]:
# get parish code poly
url = 'http://mapas.dgterritorio.pt/ATOM-download/CAOP-Cont/Cont_AAD_CAOP2020.zip'
path = 'map_data/Cont_AAD_CAOP2020'
parishes = gpd.read_file(path)
parishes.shape

(3223, 9)

# Get Polygons

In [ ]:
# make latlng readable for OSM request
res_green_areas['latlng'] = res_green_areas.apply(lambda x: (x.POINT_Y, x.POINT_X), axis=1)

def get_parks(df):
    suggestions = pd.DataFrame()
    tags = {'leisure':True}
    for c, row in df.iterrows():
        center_point = row.latlng
        park = ox.geometries.geometries_from_point(center_point, 
                                                           tags=tags, 
                                                           dist=50)
        park['match'] = c # relate the findings to the dfs index (id)
        suggestions = pd.concat([suggestions, park])
        
    return suggestions

parks = get_parks(res_green_areas)

# Merging